In [1]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor 
from pycor import utils, korlang

stopwatch = utils.StopWatch()


model_size = 7000
outputpath = "../../output/" + str(model_size) + "/"
model_path = outputpath + "model/"

pycor.loadmodel(model_path)



Init Trainer
Init DocResolver
Loading Model from ../../output/7000/model/
Load  ../../output/7000/model//heads.csv   소요시간: 0.841
Load  ../../output/7000/model//tails.csv   소요시간: 0.009


In [2]:
def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = []
files.extend( listfiles('../../data/wiki2') )
files.extend( listfiles('../../data/wiki') )
files.extend( listfiles('../samples/news') )
files.extend( listfiles('../../data/news') )
files.extend( listfiles('../../data/blotter') )
files.extend( listfiles('../../data/zdnet') )
files.extend( listfiles('../../data/ciobiz') )
files.extend( listfiles('../../data/NP') )

print (len(files), "files")

3959 files


In [3]:

yeTags = set(['EFN'])

def gettail(word):
    if word.bestpair:
#         return word.bestpair.tags
        return word.bestpair.tail.text
    return 'X'

def gettag(word):
    if word.bestpair:
        return word.bestpair.tags
    return 'X'

def gethead(word):
    if word.bestpair:
        return word.bestpair.head.text
    return 'X'


yheadsMap = {}
ytailsMap = {}
ytagsMap = {}

def handleHaed(index, word, words):
    yset = yheadsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        yheadsMap[word.bestpair.head] = yset

    if index > 1:
        yset.append( [gethead(words[index-1]),gethead(words[index-2])])
    else:
        yset.append( [gethead(words[index-1])] )
        
def handleTag(index, word, words):
    yset = ytagsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        ytagsMap[word.bestpair.head] = yset
    if index > 1:
        yset.append( [gettag(words[index-1]),gettag(words[index-2])])
    else:
        yset.append( [gettag(words[index-1])] )
                
def handleTail(index, word, words):
    yset = ytailsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        ytailsMap[word.bestpair.head] = yset
    if index > 1:
        yset.append( [gettail(words[index-1]),gettail(words[index-2])])
    else:
        yset.append( [gettail(words[index-1])] )
        
def analyzeYongEon(file):
    sentence_array, words_array = pycor.readfile(file)
    
    for words in words_array:
        for index, word in enumerate(words):
            if word.bestpair and len(yeTags & set(word.bestpair.tags))>0 and word.text.endswith('다'):
                handleHaed(index, word, words)
                handleTail(index, word, words)
                handleTag(index, word, words)


def writeFile(path, yMap):
    with open(path, 'w', encoding='utf-8') as file :
        writer = csv.writer(file)
        for head, tags in yMap.items():
            headText = head.text
            for tag in tags:
                row = [headText]
                row.extend(tag)
                writer.writerow(row)
        file.close()
                         
stopwatch.start()

docsize = 70

for file in files[:docsize]:
    if file.endswith(".txt"):
        analyzeYongEon(file)

print("Loading ",docsize," Docs: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")

Loading  70  Docs:  2s.312ms. ( 2,311 ms.)


In [4]:
# yheadsMap 
# ytailsMap  
# ytagsMap  

def fill(ylist, ymap):
    for head, tags in ymap.items():
        headText = head.text
        for tag in tags:
            row = [headText]
            row.extend(tag)
            ylist.append(row) 
        
tailsList = []
tagsList = []

fill(tailsList,ytailsMap)
fill(tagsList,ytagsMap)

josaYMap = {}
matrix = []

for index in range(len(tailsList)):
    tail = tailsList[index]
    tag = tagsList[index]
    
    text = tail[0]
    row = [text]
    row.extend(tail[1:])
    
    josaKey = ''
    for tags in tag[1:]:
        tagsStr = "+".join(tags)
        row.append(tagsStr)
        josaKey += "::"
        for t in tags:
            if t.startswith("J"):
                josaKey += "+" + t
    
    ymap = josaYMap.get(josaKey)
    if ymap is None:
        ymap = {}
        josaYMap[josaKey] = ymap
    
    count = ymap.get(text)
    if count is None:
        count = 0
    
    count += 1
    ymap[text] = count
    
    matrix.append(row)

    
def writeTailTagsFile(path, tailTagsList):
    with open(path, 'w', encoding='utf-8') as file :
        writer = csv.writer(file)
        for row in tailTagsList:
            writer.writerow(row)
                
        file.close()
        
outputpath = "../../output/" + str(docsize) + "/"

writeFile(outputpath+'/yongeon-heads.csv',yheadsMap)
# writeFile(outputpath+'/yongeon-tails.csv',ytailsMap)
# writeFile(outputpath+'/yongeon-tags.csv',ytagsMap)


writeTailTagsFile(outputpath+'/yongeon-tails-tags.csv',matrix)

print(len(matrix))

print(len(josaYMap))

1540
161


In [5]:
from pycor.res import CollocationResolver
import pycor.speechmodel as sm 

resolver = CollocationResolver()
pycor.addresolver(resolver)

def gettext(word, islast=False):
    if type(word) is sm.Sentence :
        if(word.senttype == sm.SENTENCE_TYPE_EQUIV):
            return word.text()
        else:
            return word.text()
    else:
        if islast and word.bestpair:
            return word.bestpair.head.text
        else:
            return word.text
            

sentence_array, words_array = pycor.readfile('../samples/docs/economics1.txt')

ngrams = {}

for sentence in sentence_array:
    for index in range(1, len(sentence.words)-1):
        first = gettext(sentence.words[index-1])
        second = gettext(sentence.words[index])
        second2 = gettext(sentence.words[index], True)
        third = gettext(sentence.words[index+1], True)
        
        bigram = ' '.join([first,second2])
        trigram = ' '.join([first,second,third])
        
        bigramCnt = ngrams.get(bigram)
        if bigramCnt:
            bigramCnt = bigramCnt +1
        else:
            bigramCnt = 1
        ngrams[bigram] = bigramCnt
        
        trigramCnt = ngrams.get(trigram)
        if trigramCnt:
            trigramCnt = bigramCnt +1
        else:
            trigramCnt = 1
        ngrams[trigram] = trigramCnt
#         ngrams[bigram] = [sentence.words[index-1], sentence.words[index]]
#         ngrams[trigram] = [sentence.words[index-1], sentence.words[index],sentence.words[index+1]]
#         print(first, second , third )

# for key, cnt in ngrams.items():
#     if cnt > 1:
#         print(key,cnt)

# for colloc in pycor.getmodel().collocations.values():
#     print(colloc, colloc.frequency)

In [6]:
# for file in files[:400]:
#     if file.endswith(".txt"):
#         pycor.readfile(file)
        

# print( len(pycor.getmodel().collocations))
# for colloc in pycor.getmodel().collocations.values():
#     if colloc.frequency > 3:
#         print(colloc, colloc.frequency)

In [7]:
# pycor.readfile('../samples/news/new1_hani.txt')

# for colloc in pycor.getmodel().collocations.values():
#     print(colloc, colloc.frequency)

In [8]:
from pycor.res import yongeonresolver as yr

yongeonResolver = yr.YongeonResolver()
pycor.addresolver(yongeonResolver)

for file in files[:100]:
    if file.endswith(".txt"):
        pycor.readfile(file)

yongeonResolver.writemap(outputpath + "/yongeon.csv")
# for head, pmap in yongeonResolver.phraseMap.items():
#     print(head)
    
#     for text, m in pmap.items():
#         print('  ',text)
#         for w in m:
#             if w.bestpair:
#                 print('    ', w.bestpair.tags, end=":")
#             else :
#                 print('    ', w.text, len(w.particles), end=":")
#         print()

Save  ../../output/70//yongeon.csv   소요시간: 0.025
